In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
dataset = pd.read_csv('Twitter_Data(Relabeled).csv')
dataset.columns=['clean_text','category']
dataset['category']=pd.to_numeric(dataset['category'],errors='coerce')
dataset=dataset.dropna()
dataset['category']=dataset['category'].astype(int)
train , test =train_test_split(dataset,train_size=0.7)
train.head(10)

,clean_text,category
114151,union home minister and bjp leader speaks lok ...,1
101632,five years after modi vowed clean the ganga le...,2
73111,glad that will gurgaon for every vote counts a...,2
136248,want modi will vote for bjp candidates irrespe...,1
49795,the party held back india’ space programme for...,2
141410,supporting for modi 100,2
71273,did ask what was modi doing his family wedding...,0
80846,one more train success credit railway engineer...,0
43658,heres presenting the come witness heres the of...,1
111068,great words delivered modi today with arnab,2


In [2]:
test.head(10)

,clean_text,category
140691,modi has cleaned ganga and joining rivers alre...,1
34106,louder for those the back india was never indi...,0
116114,has bankrupt minded think tank who advise path...,0
56217,you might have never seen him watching pogo bu...,1
130761,\nspeaking about notolerance policy towards co...,0
29071,recent survey teamlease services showed that m...,0
69415,reading india shoots down satellite test modi ...,0
16995,modi wawas kicked after advani its joshi turn ...,1
21909,civilizations book site troubles sharif for me...,1
104901,watch modi’ public meeting koraput odisha,1


In [3]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import torch
print(torch.cuda.is_available())
model_args = ClassificationArgs(num_train_epochs=5, max_seq_length=128,
                                train_batch_size=8, eval_batch_size=8, learning_rate=3e-5, adam_epsilon=1e-08,
                                use_multiprocessing=False,
                                output_dir='./outputs/', overwrite_output_dir=True,
                                save_model_every_epoch=True)
model = ClassificationModel(
    'xlnet', 'xlnet-base-cased', num_labels=3, args=model_args, use_cuda=True)


True


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [4]:
model.train_model(train, use_cuda=True, verbose=True)

/home/rafxar/Proyecto-Final-De-Carrera-1/BERT/bert-env/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:585: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
Epoch 1 of 5:   0%|          | 0/5 [00:00<?, ?it/s]/home/rafxar/Proyecto-Final-De-Carrera-1/BERT/bert-env/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch 5 of 5: 100%|██████████| 5/5 [3:17:44<00:00, 2372.95s/it]


(71300, 0.1710459726249143)

In [11]:
from sklearn.metrics import accuracy_score
results, model_outputs, wrong_predictions = model.eval_model(
    test, acc=accuracy_score)
print(results)


/home/rafxar/Proyecto-Final-De-Carrera-1/BERT/bert-env/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:1426: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Running Evaluation: 100%|██████████| 6112/6112 [04:37<00:00, 22.06it/s]

{'mcc': 0.968946563736971, 'acc': 0.98009858665194, 'eval_loss': 0.11044914731885801}
